# My first basic server

---

``class BaseHTTPRequestHandler()``
HTTP request handler base class.

- The following explanation of HTTP serves to guide you through the code as well as to expose any misunderstandings I may have about HTTP (so you don't need to read the code to figure out I'm wrong
:-).

- HTTP (HyperText Transfer Protocol) is an extensible protocol on top of a reliable stream transport (e.g. TCP/IP). The protocol recognizes three parts to a request:

One line identifying the request type and path
An optional set of RFC-822-style headers
An optional data part
The headers and data are separated by a blank line.

- The first line of the request has the form

```<command> <path> <version>```

where `<command>` is a (case-sensitive) keyword such as GET or POST, `<path>` is a string containing path information for the request, and `<version>` should be the string "HTTP/1.0" or "HTTP/1.1". `<path>` is encoded using the URL encoding scheme (using %xx to signify the ASCII character with hex code xx).

The specification specifies that lines are separated by CRLF but for compatibility with the widest range of clients recommends servers also handle LF. Similarly, whitespace in the request line is treated sensibly (allowing multiple spaces between components and allowing trailing whitespace).

Similarly, for output, lines ought to be separated by CRLF pairs but most clients grok LF characters just fine.

If the first line of the request has the form

> `<command>` `<path>`

(i.e. `<version>` is left out) then this is assumed to be an HTTP 0.9 request; this form has no optional headers and data part and the reply consists of just the data.

- The reply form of the HTTP 1.x protocol again has three parts:

One line giving the response code
An optional set of RFC-822-style headers
The data
Again, the headers and data are separated by a blank line.

- The response code line has the form

> `<version> <responsecode> <responsestring>`

where `<version>` is the protocol version ("HTTP/1.0" or "HTTP/1.1"), `<responsecode>` is a 3-digit response code indicating success or failure of the request, and `<responsestring>` is an optional human-readable string explaining what the response code means.

This server parses the request and the headers, and then calls a function specific to the request type (`<command>`). Specifically, a request SPAM will be handled by a method do_SPAM(). If no such method exists the server sends an error response to the client. If it exists, it is called with no arguments:

do_SPAM()

Note that the request name is case sensitive (i.e. SPAM and spam are different requests).

The various request details are stored in instance variables:

client_address is the client IP address in the form (host, port);

command, path and version are the broken-down request line;

headers is an instance of email.message.Message (or a derived class) containing the header information;

rfile is a file object open for reading positioned at the start of the optional input data part;

wfile is a file object open for writing.

IT IS IMPORTANT TO ADHERE TO THE PROTOCOL FOR WRITING!

The first thing to be written must be the response line. Then follow 0 or more header lines, then a blank line, and then the actual data (if any). The meaning of the header lines depends on the command executed by the server; in most cases, when data is returned, there should be at least one header line of the form

Content-type: `<type>`/`<subtype>`

where `<type>` and `<subtype>` should be registered MIME types, e.g. "text/html" or "text/plain".


In [10]:
from http.server import BaseHTTPRequestHandler, HTTPServer

In [11]:
# Definindo uma classe herdeira de BaseHTTPRequestHandler
class SimpleHTTPRequestHandler(BaseHTTPRequestHandler):
    
	# Lidar com requisições GET
    def do_GET(self):
        
		# Envia status code 200 ( OK )
        self.send_response(200)
        
		# Define cabeçalho da resposta
        self.send_header('Content-type', 'text/html')
        self.end_headers()
        
        # Resposta ao client
        resposta = "<h1>Olá, Mundo, de Jhonathan!</h1>"
        
        self.wfile.write(resposta.encode('utf-8'))
	
# Função para rodar o servidor
def run(server_class=HTTPServer, handler_class=SimpleHTTPRequestHandler, port=8000):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f'Servidor rodando na porta {port}...')
    httpd.serve_forever()

# Rodando o servidor
if __name__ == "__main__":
    run()

Servidor rodando na porta 8000...


127.0.0.1 - - [14/Jan/2025 00:08:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Jan/2025 00:08:40] "GET /favicon.ico HTTP/1.1" 200 -


KeyboardInterrupt: 